In [124]:
import pandas
import re

In [125]:
ground = pandas.read_json('data/input/draw.json')
ground = ground[['lSolutions', 'iIndex']]
ground.head(1)

,lSolutions,iIndex
0,[2.14285714286],397760


In [126]:
solved = pandas.read_json('data/output/solved/sample_0.jsonl', lines=True)
solved.head(1)

,question_id,id,object,created,model,usage,choices,question,n,temperature,solved,flag
0,113024,chatcmpl-7NHX754J9stbIGK5w2pqoUuT8P0nM,chat.completion,1685783297,gpt-3.5-turbo,"{'prompt_tokens': 407, 'completion_tokens': 11...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,"{a: 20, b: 4}",OK


In [127]:
joined = pandas.merge(left=solved, right=ground, left_on='question_id', right_on='iIndex')
joined.head(1)

,question_id,id,object,created,model,usage,choices,question,n,temperature,solved,flag,lSolutions,iIndex
0,113024,chatcmpl-7NHX754J9stbIGK5w2pqoUuT8P0nM,chat.completion,1685783297,gpt-3.5-turbo,"{'prompt_tokens': 407, 'completion_tokens': 11...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,"{a: 20, b: 4}",OK,"[20.0, 4.0]",113024


In [128]:
def extract_actual_answer(answer):
    answer = re.sub(r'\s', "", answer)
    answer = re.sub(r'\s|.*####', "", answer)
    return eval(answer)

def extract_chatgpt_answer(answer):
    answer = answer.lower()
    try:
        answer = re.findall(r'the\s*answer\s*is\s*.*', answer)[0]
        answer = re.findall(r'\d+\.?\d*', answer)
        return eval(answer)
    except:
        answer = re.findall(r'\d+\.?\d*', answer)
        if len(answer) == 0: return "WRONG"
        else: return eval(answer)

def extract_solved(solved):
    return set(re.findall(r'the\s*answer\s*is\s*.*', solved))

def grade_response(row):
    solved = row['solved']
    chatgpt_answer = round(extract_chatgpt_answer(row['question']), 2)
    actual_answer = round(extract_actual_answer(row['answer']), 2)

    A = 1 if chatgpt_answer == actual_answer else 0
    B = 
    C = 
    D = 
    pass

SyntaxError: invalid syntax (2360790856.py, line 26)

In [ ]:
extract_actual_answer("""
Simplifying:

x = 540 meters/week

Step 2: Final answer
The answer is 540 meters per week.



He sprints 3*3=<<3*3=9>>9 times
So he runs 9*60=<<9*60=540>>540 meters
#### 540
""")

540

In [ ]:
extract_chatgpt_answer("""
Step 1: Algebraic system of equations
Let x be the number of meters James runs in a week.
Each sprint is 60 meters, and he runs 3 sprints 3 times a week, so:

x = 60 meters/sprint * 3 sprints/time * 3 times/week

Simplifying:

x = 540 meters/week

Step 2: Final answer
The answer is 540 meters per week.

""")

540